In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from Energy.HelpFunctions.get_energy_data import get_energy_data, prepare_data

In [28]:
df = get_energy_data()
df = prepare_data(df)

  0%|          | 0/255 [00:00<?, ?it/s]/Users/elias/Desktop/PTSFC/Energy/HelpFunctions/get_energy_data.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  energydata = pd.concat([energydata, pd.DataFrame(rawdata, columns=col_names)])
100%|██████████| 255/255 [00:27<00:00,  9.25it/s]


In [71]:
def split_time(df, num_years = 0, num_months = 0, num_weeks = 0, num_days = 0, num_hours = 0):
    # date_to_keep = df.index.max() - num_years - num_months - num_days
    split_date = df.index.max() - pd.DateOffset(years = num_years, months=num_months, weeks = num_weeks, days=num_days, hours = num_hours)
    df_b = df.loc[df.index <= split_date]
    df_a = df.loc[df.index > split_date]
    
    return df_b, df_a

In [30]:
df

,gesamt,weekday
date_time,,
2018-12-24 00:00:00,42.02925,0
2018-12-24 01:00:00,39.61025,0
2018-12-24 02:00:00,39.13875,0
2018-12-24 03:00:00,39.42100,0
2018-12-24 04:00:00,40.74775,0
...,...,...
2023-11-12 03:00:00,38.68725,6
2023-11-12 04:00:00,39.28950,6
2023-11-12 05:00:00,39.31125,6


In [44]:

from Energy.Baseline.baseline import baseline

baseline(df, LAST_IDX=-1)

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2023-11-13 20:00:00,energy,36 hour,47.099250,54.790000,58.275750,61.938625,68.925319
1,2023-11-14 00:00:00,energy,40 hour,39.612500,43.428562,45.941125,49.032313,56.291488
2,2023-11-14 04:00:00,energy,44 hour,39.472119,41.745313,44.774500,48.597875,55.372306
3,2023-11-14 20:00:00,energy,60 hour,51.757831,55.769188,58.404375,62.398563,69.663931
4,2023-11-15 00:00:00,energy,64 hour,41.628875,44.106687,45.769250,49.245688,56.137538
5,2023-11-15 04:00:00,energy,68 hour,40.345519,42.847750,44.830875,48.450500,55.380406


# Cross validate

In [32]:
from datetime import datetime, timedelta


def most_recent_thursday(df):
    today = df.index.max()
    days_to_thursday = (today.weekday() - 3) % 7  # Calculate days to the most recent Thursday
    recent_thursday = today - timedelta(days=days_to_thursday, hours=today.hour, minutes=today.minute, seconds=today.second, microseconds=today.microsecond)
    
    # Set the time to 12:00 AM
    most_recent_thursday = recent_thursday.replace(hour=0, minute=0, second=0, microsecond=0)
    
    return most_recent_thursday




Remove everything til last thursday night 12pm

In [66]:
start_date_excl = most_recent_thursday(df)
df_cval = df.loc[df.index < start_date_excl]

In [67]:
baseline(df_cval, LAST_IDX=-1)

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2023-11-10 12:00:00,energy,36 hour,53.274763,60.872563,63.616750,67.128563,74.963481
1,2023-11-10 16:00:00,energy,40 hour,50.642019,55.814688,58.537500,62.717125,71.090000
2,2023-11-10 20:00:00,energy,44 hour,49.348806,52.384937,56.379125,60.121625,67.135544
3,2023-11-11 12:00:00,energy,60 hour,47.477200,51.317125,53.837750,57.298625,63.596281
4,2023-11-11 16:00:00,energy,64 hour,44.081156,47.382188,50.045750,53.573437,62.625650
5,2023-11-11 20:00:00,energy,68 hour,43.284444,46.577937,49.405625,53.355125,59.913819


Repeatedly run the model. Record predictions and true values (observations). Make sure the observations are available for the most recent prediction.

In [161]:
from Energy.HelpFunctions.calc_score import evaluate_horizon

# import importlib
# importlib.reload(Energy.Baseline)

evaluation = pd.DataFrame()
df_before = df_cval
for w in range(100):
    # Create a temp df that only contains the values used for training
    df_before, df_after = split_time(df_before, num_weeks=1)
    pred = baseline(df_before, LAST_IDX=-1)
    # Transform the observations to timeseries with same index and then merge
    obs = pd.DataFrame({'gesamt': df_cval.loc[pred['forecast_date']]["gesamt"]})
    obs = obs.rename_axis('forecast_date')
    pred = pred.set_index('forecast_date')
    # print(pred.iloc[0]['q0.025','q0.25'])
    merged_df = pd.merge(pred, obs, left_index=True, right_index=True) 
    
    # Add scores to the merged_df
    for index, row in merged_df.iterrows():
        quantile_preds = row[['q0.025','q0.25','q0.5','q0.75','q0.975']]
        observation = row['gesamt']
        score = evaluate_horizon(quantile_preds, observation)
        merged_df.at[index, 'score'] = score
    # print(merged_df[['q0.025','q0.25','q0.5','q0.75','q0.975']])
    evaluation = pd.concat([evaluation, merged_df])
    

Calculate score for: q = 53.2747625, tau = 0.025: 0.6339618749999997
Calculate score for: q = 60.8725625, tau = 0.25: 2.5407187499999964
Calculate score for: q = 63.616749999999996, tau = 0.5: 2.3372499999999974
Calculate score for: q = 67.4963125, tau = 0.75: 0.7711562500000042
Calculate score for: q = 75.3257, tau = 0.975: 0.46858500000000064
Calculate score for: q = 50.642018750000005, tau = 0.025: 0.5422865624999996
Calculate score for: q = 55.814687500000005, tau = 0.25: 2.8365312499999966
Calculate score for: q = 58.5375, tau = 0.5: 2.950249999999997
Calculate score for: q = 63.0749375, tau = 0.75: 0.7935937499999994
Calculate score for: q = 71.60369374999999, tau = 0.975: 0.5057971875
Calculate score for: q = 49.348806249999996, tau = 0.025: 0.4634846875000001
Calculate score for: q = 52.3849375, tau = 0.25: 3.116781249999999
Calculate score for: q = 56.379125, tau = 0.5: 2.2393749999999955
Calculate score for: q = 60.4970625, tau = 0.75: 0.9392812500000005
Calculate score for: 

In [162]:
evaluation

,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975,gesamt,score
forecast_date,,,,,,,,,
2023-11-03 12:00:00,energy,36 hour,53.274763,60.872563,63.616750,67.496313,75.325700,65.95400,6.751672
2023-11-03 16:00:00,energy,40 hour,50.642019,55.814688,58.537500,63.074937,71.603694,61.48775,7.628459
2023-11-03 20:00:00,energy,44 hour,49.348806,52.384937,56.379125,60.497062,67.426706,58.61850,7.199332
2023-11-04 12:00:00,energy,60 hour,47.477200,51.317125,53.837750,57.682688,63.793750,55.63325,5.794109
2023-11-04 16:00:00,energy,64 hour,44.081156,47.382188,50.045750,53.573437,63.037831,54.13450,9.254334
...,...,...,...,...,...,...,...,...,...
2021-12-10 16:00:00,energy,40 hour,49.422063,58.959750,61.605000,66.756438,72.169269,72.00400,25.929829
2021-12-10 20:00:00,energy,44 hour,49.794556,55.053125,59.819375,63.881313,66.994919,65.66425,14.684862
2021-12-11 12:00:00,energy,60 hour,51.602300,54.199938,55.887000,59.868625,63.831612,63.54050,18.443059


In [92]:
df_cval

,gesamt,weekday
date_time,,
2018-12-24 00:00:00,42.02925,0
2018-12-24 01:00:00,39.61025,0
2018-12-24 02:00:00,39.13875,0
2018-12-24 03:00:00,39.42100,0
2018-12-24 04:00:00,40.74775,0
...,...,...
2023-11-08 19:00:00,66.99950,2
2023-11-08 20:00:00,64.17500,2
2023-11-08 21:00:00,60.00300,2
